# Extract and store the stimulaiton data and an example trace
The files *['localhost-20230824_115343', 'localhost-20230905_131313', 'localhost-20230905_115224']* must be in a *./data directory* relative to this notebook.

Author: Bjarne Schultze <br>
Last modified: 03.12.2023

In [ ]:
import numpy as np
import h5py
import opto_analysis_fun as oafun

In [ ]:
# List the files to use for plotting the stimulation protocols
files = ['localhost-20230824_115343','localhost-20230905_131313','localhost-20230905_115224']  # red (high intensities), red (low intensities), green light (high intensities)
# Load the calibration table for the LEDs and extract information for red and green light
opto_cal = np.loadtxt('opto_calibration.txt', skiprows=1)
cal_volt = opto_cal[:,0]
stim_red = opto_cal[:,1:4].mean(axis=1)
stim_green = opto_cal[:,4:].mean(axis=1)

# Define list to collect the stimulation protocols
stim_vec = []

# Iterate over the given files
for file in files:
    # Load the current dataset
    data_in = h5py.File('./data/'+file+'/'+file+'_daq.h5')
    print(f'Working on file: {file}')
    dataset = np.array(data_in['samples'])

    # Get the sampling rate
    sampling_rate = round(np.mean(data_in['samplenumber']))
    # Create a time vector in minutes
    time_vec = np.arange(0, dataset.shape[0]/sampling_rate, 1/sampling_rate) / 60

    # Store the data of an example trace separately
    if file == 'localhost-20230824_115343':
        exp_data = dataset
        exp_time = time_vec

    # Close data file connection
    data_in.close()

    # Extract and analyze the light stimulus
    light, stim_mode, stim_on, stim_off, stim_volt = oafun.analyze_light_stim(dataset[:,3:])

    # Array with zeros to be filled with the light intensities (according to the calibration)
    stim_intensity = np.zeros_like(light)
    # Iterate over all stimuli (5 s pulses)
    for on, off, volt in zip(stim_on, stim_off, stim_volt):
        # Find the stimulus voltage among the calibration values
        find_stim_volt = cal_volt == volt
        # Fill the new array with the according light intensity 
        if stim_mode == 'activation':
            stim_intensity[on:off] = stim_red[find_stim_volt]
        elif stim_mode == 'inactivation':
            stim_intensity[on:off] = stim_green[find_stim_volt]
    
    # Store the stimulus array (light intensities)
    stim_vec.append(stim_intensity)

In [ ]:
# Write data to hdf5 file for later usage
f = h5py.File('stim_exp_data.hdf5', 'a')

# Store the data
f['stim_vec'] = stim_vec
f['stim_time'] = time_vec
f['example_trace'] = exp_data
f['example_time'] = exp_time

# Close the output file
f.close()